# Look at barcode diversity in real life

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import iss_preprocess as issp
from iss_analysis.barcodes import barcodes as bar
from iss_analysis.barcodes.diagnostics import plot_gmm_clusters, plot_error_along_sequence
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Get data

## Get all the barcode and filter with GMM

In [ ]:
data_path = "becalia_rabies_barseq/BRAC8498.3e"
analysis_folder = issp.io.get_processed_path(data_path) / "analysis"
analysis_folder.mkdir(exist_ok=True)


In [ ]:

thresholds = dict(mean_intensity=0.01, dot_product_score=0.2, mean_score=0.5)
barcode_spots, gmm, all_barcode_spots = bar.get_barcodes(
    acquisition_folder=data_path,
**{f"{k}_threshold": v for k, v in thresholds.items()})

pg = plot_gmm_clusters(all_barcode_spots, gmm, thresholds=thresholds)
fig = pg.figure
fig.savefig(analysis_folder / "gmm_clusters.png")


## Plot proportion of barcodes in each cluster

In [ ]:

ch_r_gp = all_barcode_spots.groupby(['chamber', 'roi'])
n_rois = len(ch_r_gp)
n_per_lab = np.zeros((n_rois, 3))
for i, (ch_r, df) in enumerate(ch_r_gp):
    n_per_lab[i] = df.groupby('gmm_label').size()

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(2,1,1)
ax.plot(np.arange(n_rois), n_per_lab, 'o-')
ax.set_xticks(np.arange(n_rois))
ax.set_xticklabels([])
ax.set_ylabel('Number of spots')
ax = fig.add_subplot(2,1,2)
ax.plot(np.arange(n_rois), n_per_lab/np.sum(n_per_lab, axis=1, keepdims=True), 'o-')
ax.set_xticks(np.arange(n_rois))
ax.set_ylabel('Fraction of spots')
_ = ax.set_xticklabels([f"{ch}_{r}" for ch, r in ch_r_gp.groups.keys()], rotation=90)
fig.tight_layout()

## Plot one example chamber

In [ ]:
chamber = 8
roi = 8

import matplotlib.pyplot as plt
df = all_barcode_spots[(all_barcode_spots["chamber"] == f"chamber_{chamber:02}") & (all_barcode_spots["roi"] == roi)]
fig = plt.figure(figsize=(20, 10), facecolor='black')
ax = fig.add_subplot(1,1,1,aspect='equal', facecolor='black')
for i in range(3):
    ax.scatter(df[df["gmm_label"] == i]["y"], df[df["gmm_label"] == i]["x"], label=f"cluster {i}",
                alpha=0.1, s=1)
plt.legend()
_ = ax.axis('off')

# Error correction

## Estimate error rate along sequence

In [ ]:
errors_along_seq = bar.error_per_round(spot_df=barcode_spots, edit_distance=1, spot_count_threshold=30, sequence_column='sequence', filter_column='bases')

In [ ]:
figs = plot_error_along_sequence(errors_along_seq, nrows=2, plot_matrix=False, marker='o')

## Manually filter out missing data

In [ ]:
# This is INDEX of the round, NOT the number as written in the file name, so 0 based.
missing_roi = dict(chamber_09={8: 2, 9: 2}, chamber_08={8:10, 9:10})
missing_tiles = dict(chamber_08={'10_2_4':5})

In [ ]:
clean_barcodes = barcode_spots.copy()

print('Removing rounds with missing rois')
for chamber, rois in missing_roi.items():
    for roi, round_ in rois.items():
        print(f"Removing round {round_} from {chamber}, roi {roi}")
        bad = (clean_barcodes["chamber"] == chamber) & (clean_barcodes["roi"] == roi)
        mask = np.zeros(len(clean_barcodes.iloc[0]['sequence']), dtype=float)
        mask[round_] = np.nan
        c = clean_barcodes.loc[bad, 'sequence'].map(lambda x: x + mask)
        clean_barcodes.loc[bad, 'sequence'] = c
        c = clean_barcodes.loc[bad, 'bases'].map(lambda x: x[:round_] + 'N' + x[round_ + 1:])
        clean_barcodes.loc[bad, 'bases'] = c
print('Removing rounds with missing tiles')
for chamber, tiles in missing_tiles.items():
    for tile, round_ in tiles.items():
        print(f"Removing round {round_} from {chamber}, tile {tile}")
        bad = (clean_barcodes["chamber"] == chamber) & (clean_barcodes["tile"] == tile)
        mask = np.zeros(len(clean_barcodes.iloc[0]['sequence']), dtype=float)
        mask[round_] = np.nan
        c = clean_barcodes.loc[bad, 'sequence'].map(lambda x: x + mask)
        clean_barcodes.loc[bad, 'sequence'] = c
        c = clean_barcodes.loc[bad, 'bases'].map(lambda x: x[:round_] + 'N' + x[round_ + 1:])
        clean_barcodes.loc[bad, 'bases'] = c

In [ ]:
errors_along_seq = bar.error_per_round(spot_df=clean_barcodes, edit_distance=1, spot_count_threshold=30)
figs = plot_error_along_sequence(errors_along_seq, nrows=2, plot_matrix=False, marker='o')

## Actual error correction part

In [ ]:
import pandas as pd
redo = False
weights = np.ones(14)
# weights[10:] = 0
err_corr_edt = dict()
for edist in [3]:
    print(f"Correcting barcodes with edit distance {edist}")
    fname = analysis_folder / f"corrected_barcode_spots_edit{edist}.pkl"
    if redo or not fname.exists():
        err_corr, merge_dict = bar.correct_barcode_sequences(clean_barcodes, max_edit_distance=edist, weights=weights, return_merge_dict=True)
        err_corr.to_pickle(fname)
    else:
        err_corr = pd.read_pickle(fname)
    non_corr = err_corr.bases.value_counts()
    corr = err_corr.corrected_bases.value_counts()
    print(f"Number of unique sequences before correction: {len(non_corr)}")
    print(f"Number of unique sequences after correction: {len(corr)}")
    err_corr_edt[edist] = err_corr



## Quality control

In [ ]:
for threshold in range(10, 100, 10):
    n_bc = {}
    n_bc_with_N = {}
    for k, v in err_corr_edt.items():
        v['corrected_bases'] = v['corrected_bases'].map(lambda x: ''.join(x))
        vc = v['corrected_bases'].value_counts()
        n_bc[k] = vc[vc > threshold].shape[0]
        n_bc_with_N[k] = np.sum(['N' in i for i in vc.index])
    plt.plot(n_bc.keys(), n_bc.values(), 'o-', label=f"Aboundance > {threshold}")
plt.ylabel('Number of unique barcodes')
plt.xlabel('Edit distance')
plt.legend()

In [ ]:
ab = vc[vc >= 500]

nchamber = len(v.chamber.unique())
nrois= 10
fig, axes = plt.subplots(nrois, nchamber, figsize=(20, 40))
for bci in range(5):
    bc = v[v.corrected_bases==ab.index[bci]]
    for ich, (ch_name, ch) in enumerate(v.groupby('chamber')):
        for iroi, (roi_name, roi) in enumerate(ch.groupby('roi')):
            ax = axes[iroi, ich]
            bcc = bc[(bc.roi == roi_name) & (bc.chamber == ch_name)]
            if not bci:
                ax.scatter(roi['y'], roi['x'], label=f"roi {iroi}", s=1, color='k', alpha=0.05)
            ax.scatter(bcc['y'], bcc['x'], label=f"roi {iroi}", s=5, color=f'C{bci}', alpha=0.5)
            ax.axis('equal')
            ax.axis('off')
fig.tight_layout()

In [ ]:


plt.plot(n_bc_with_N.keys(), n_bc_with_N.values(), 'o-', label="with N")
plt.ylabel('Number of unique barcodes')
plt.xlabel('Edit distance')
plt.legend()

In [ ]:
nan_to_start = np.sum(['N' in i for i in non_corr.index])
nan_after = np.sum(['N' in i for i in corr.index])
print(f"Number of 'unique' sequences with N before correction: {nan_to_start}")
print(f"Number of 'unique' sequences with N after correction: {nan_after}")
print(f"Number of sequences with N corrected: {nan_to_start-nan_after}")

In [ ]:
fig = plt.figure(figsize=(7, 3)) 
fig.suptitle("# Rolonies per sequence")
ax = fig.add_subplot(1, 2, 1)
ax.hist(non_corr.values, bins=np.arange(0,corr.max(),10), log=True, histtype='step', label='non-corrected')
ax.hist(corr.values, bins=np.arange(0,corr.max(),10), log=True, histtype='step', label='corrected')
ax = fig.add_subplot(1, 2, 2)
ax.hist(non_corr.values, bins=np.arange(100), log=True, label='non-corrected', histtype='step')
ax.hist(corr.values, bins=np.arange(100), log=True, label='corrected', histtype='step')
ax.legend(loc='upper right')
fig.tight_layout()

In [ ]:
fig = plt.figure(figsize=(7, 3)) 
fig.suptitle("# Rolonies per sequence")
ax = fig.add_subplot(1, 2, 1)
ax.hist(non_corr.values, bins=np.arange(corr.max()), log=False, histtype='step', label='non-corrected', cumulative=True, density=True)
ax.hist(corr.values, bins=np.arange(corr.max()), log=False, histtype='step', label='corrected', cumulative=True, density=True)
ax = fig.add_subplot(1, 2, 2)
ax.hist(non_corr.values, bins=np.arange(100), log=False, label='non-corrected', histtype='step', density=True)
ax.hist(corr.values, bins=np.arange(100), log=False, label='corrected', histtype='step', density=True)
ax.legend(loc='upper right')
fig.tight_layout()

In [ ]:
from iss_preprocess.call import BASES
th = 1
good = corr[corr > 30].index
print(len(good))
sequences = np.stack(err_corr["corrected_sequence"].to_numpy())
error_along_sequence = np.zeros((len(good), sequences.shape[1]))
bases = list(BASES) + ['N']
for ibar, barcode in enumerate(good):
    seq = [bases.index(b) for b in barcode]
    diff = sequences - seq
    edit_distance = np.sum(diff != 0, axis=1)
    actual_errs = edit_distance <= th
    bad_barcode = diff[actual_errs]
    error_along_sequence[ibar] = np.any(bad_barcode != 0, axis=0)


In [ ]:
def plot_error_along_seq(error_along_sequence):
    fig = plt.figure(figsize=(10, 5))
    fig.suptitle("Errors along sequence (edit distance 1)")
    ax = fig.add_subplot(1, 2, 1)
    im = ax.imshow(error_along_sequence, aspect='auto')
    ax.set_xlabel('Position')
    ax.set_ylabel('Barcode')
    cb = fig.colorbar(im, ax=ax)
    cb.set_label('Errors')
    ax = fig.add_subplot(1, 2, 2)
    ax.plot(np.mean(error_along_sequence, axis=0))
    ax.set_xlabel('Position')
    ax.set_ylabel('Mean errors')
    fig.tight_layout()
    return fig
f = plot_error_along_seq(error_along_sequence)

In [ ]:
non_corr = err_corr.bases.value_counts()
corr = err_corr.corrected_bases.value_counts()
df_non_corr = pd.DataFrame(dict(seq=non_corr.index, cnt=non_corr.values))
df_corr = pd.DataFrame(dict(seq=corr.index, cnt=corr.values))

print(df_non_corr.head())


In [ ]:
ok = df_non_corr.cnt > 50
print(df_non_corr[ok].cnt.sum() / df_non_corr.cnt.sum())
ok = df_corr.cnt > 50
print(df_corr[ok].cnt.sum() / df_corr.cnt.sum())

In [ ]:
fig = plt.figure(figsize=(5, 3))
ax = fig.add_subplot(1,1,1)
ax.plot(np.log(df_non_corr.index),np.log(df_non_corr["cnt"]), label='non-corrected')
ax.plot(np.log(df_corr.index), np.log(df_corr["cnt"]), label='corrected')
plt.legend()
ax.set_xlabel('Rank')
ax.set_ylabel('log(count)')


In [ ]:
all_vs.shape

In [ ]:
fig = plt.figure(figsize=(10, 3))
print(all_vs.max())
ax = fig.add_subplot(1, 2, 1)
_ = ax.hist(all_vs, bins=np.arange(0, all_vs.max(), 10), log=True)
ax.text(800, 10e3, f"Max: {all_vs.max():.0f}")
ax.set_ylabel('Number of rolonies before merge')
ax = fig.add_subplot(1, 2, 2)
_ = ax.hist(all_vs, bins=np.arange(25) + 0.5, log=False)
ax.set_xticks(np.arange(0, 24, 2) + 1)
ax.set_ylabel('Number of rolonies before merge')
fig.tight_layout()


# Check registration

Save overview plot to check registration

In [ ]:
# helper plotting functions
def get_stack_part(stack, xlim, ylim):
    ylim = sorted(ylim)
    xlim = sorted(xlim)
    return stack[ylim[0]:ylim[1], xlim[0]:xlim[1]]

def get_spot_part(df, xlim, ylim):
    ylim = sorted(ylim)
    xlim = sorted(xlim)
    return df[(df["x"] >= xlim[0]) & (df["x"] < xlim[1]) & (df["y"] >= ylim[0]) & (df["y"] < ylim[1])]

In [ ]:
def get_stacks(chamber, roi, data_path):
    ref = issp.pipeline.stitch_registered(data_path + f"/{chamber}", prefix='hybridisation_round_2_1', roi=roi, channels=[2,3])
    #barcode_round_1 = issp.pipeline.stitch_registered(data_path + f"/{chamber}", prefix='barcode_round_1_1', roi=roi, channels=range(4))
    #barcode_round_1 = np.nanmax(barcode_round_1, axis=2)
    barcode_round_1 = None
    mcherry = issp.pipeline.stitch_registered(data_path + f"/{chamber}", prefix='mCherry_1', roi=roi, channels=2)
    return ref, barcode_round_1, mcherry

chamber= 'chamber_09'
roi = 3
ref = issp.pipeline.stitch_registered(data_path + f"/{chamber}", prefix='hybridisation_round_2_1', roi=roi, channels=[2,3])
mcherry = issp.pipeline.stitch_registered(data_path + f"/{chamber}", prefix='mCherry_1', roi=roi, channels=2)



In [ ]:
barcode_round_1 = issp.pipeline.stitch_registered(data_path + f"/{chamber}", prefix='barcode_round_1_1', roi=roi, channels=range(4))

In [ ]:
def get_st(data_path, roi):
    ops = issp.io.load_ops(data_path)
    ref_prefix = ops['reference_prefix']
    ref_ch = ops['reg2ref_reference_channels']
    print(f"Stitching ref")
    ref = issp.pipeline.stitch_registered(data_path, prefix=ref_prefix, roi=roi, channels=ref_ch)

    bc_chans = ops.get('reg2ref_barcode_channels', [0, 1, 2, 3])
    print(f"Stitching barcode")
    barcode_round_1 = issp.pipeline.stitch_registered(data_path, prefix='barcode_round_1_1', roi=roi, channels=bc_chans)
    barcode_round_1 = np.nanmax(barcode_round_1, axis=2)
    mch_chan = ops.get('reg2ref_mCherry_channels', [3])
    print(f"Stitching mCherry")
    mcherry = issp.pipeline.stitch_registered(data_path, prefix='mCherry_1', roi=roi, channels=mch_chan)
    return ref, barcode_round_1, mcherry
print(chamber, roi)
ref, barcode_round_1, mcherry = get_st(data_path + f"/{chamber}", roi)


In [ ]:
def get_stack_part(stack, xlim, ylim):
    ylim = sorted(ylim)
    xlim = sorted(xlim)
    return stack[ylim[0]:ylim[1], xlim[0]:xlim[1]]

st = np.dstack([ref[...,0], mcherry,  barcode_round_1])
rgb = issp.vis.to_rgb(st, colors=[(0, 0, 1), (1, 0, 0), (0, 1, 0)], 
                        vmin=np.nanpercentile(st, 1, axis=(0,1)), vmax=np.nanpercentile(st, 99.9, axis=(0,1)))
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(2, 1, 1)
ax.imshow(rgb, interpolation='none')
ax.set_axis_off()

w, h = ref.shape[:2]
for ix in range(2):
    for iy in range(2):
        ax = fig.add_subplot(4, 2, ix + 2 * iy + 5)
        s_corner = get_stack_part(st, [ix * w // 4, (ix + 1) * w // 4], [iy * h // 4, (iy + 1) * h // 4])
        rgb = issp.vis.to_rgb(s_corner, colors=[(0, 0, 1), (1, 0, 0), (0, 1, 0)], 
                        vmin=np.nanpercentile(s_corner, 1, axis=(0,1)), vmax=np.nanpercentile(s_corner, 99.9, axis=(0,1)))
        ax.imshow(rgb, interpolation='none')
        ax.set_axis_off()

# Look at starters

## Select example ROI

In [ ]:
# try to make a big registered mask image
chamber= 'chamber_08'
roi = 4
print(data_path + f"/{chamber}")


In [ ]:
st = np.dstack([ref[...,1], mcherry, np.nanmax(barcode_round_1, axis=2)])
rgb = issp.vis.to_rgb(st, colors=[(0, 0, 1), (1, 0, 0), (0, 1, 0)], vmin=np.nanpercentile(st, 1, axis=(0,1)), vmax=np.nanpercentile(st, 99.9, axis=(0,1)))
plt.figure(figsize=(20, 20))
plt.imshow(rgb, interpolation='none')

## Segment rolonies

In [ ]:
1+1

In [ ]:
from pathlib import Path

chamber = 'chamber_10'
slurm_folder = Path.home() / "slurm_logs" /( data_path + f"/{chamber}") / "align_spots"
slurm_folder.mkdir(parents=True, exist_ok=True)
ref_job_id = None

print(data_path + f"/{chamber}")
ops = issp.io.load_ops(data_path + f"/{chamber}",)
ref_prefix = ops["reference_prefix"]
ref_job_id = issp.pipeline.stitch.register_within_acquisition(
    data_path = data_path + f"/{chamber}",
    prefix=ref_prefix,
    reload=True,
    save_plot=True,
    use_slurm=False,
    slurm_folder=slurm_folder,
    scripts_name=f"register_within_acquisition_{ref_prefix}",
)

In [ ]:
from atexit import register
from pathlib import Path
pathc = data_path + f"/chamber_10"
for roi in range(1, 11):
    issp.pipeline.register.register_to_ref_using_stitched_registration(
        pathc,
        reg_prefix='mCherry_1',
        ref_prefix=None,
        rois=[roi],
        ref_channels=None,
        reg_channels=None,
        estimate_rotation=True,
        target_suffix=None,
        use_masked_correlation=False,
        downsample=3,
        use_slurm=True,
        slurm_folder=Path.home() / "slurm_logs" / pathc,
        scripts_name=f"register_to_ref_{roi}",
    )

In [ ]:
df = err_corr[(err_corr["chamber"] == chamber) & (err_corr["roi"] == roi)].copy()
barcode_in_cells = issp.segment.cells.count_spots(df, grouping_column='corrected_bases', masks=bigmask)
all_cells = np.unique(bigmask)
missing = set(all_cells) - set(barcode_in_cells.index)
missing_df = pd.DataFrame(index=list(missing), columns=barcode_in_cells.columns, 
data=np.zeros((len(missing), len(barcode_in_cells.columns))))
barcode_in_cells = pd.concat([barcode_in_cells, missing_df])

In [ ]:
edist = 3
err_corr = pd.read_pickle(analysis_folder / f"corrected_barcode_spots_edit{edist}.pkl")

In [ ]:
from pathlib import Path
import iss_analysis as issa
redo = False
skip_not_done = False
use_slurm = True
all_cells_list = []
edist = 3
for chamber in [f"chamber_{i:02}" for i in [8,9, 10]]: #err_corr.chamber.unique():
    if chamber == 'chamber_07':
        continue
    print(f'Doing {chamber}')
    for roi in range(1, 11): # err_corr.roi.unique():
        target = analysis_folder / f"barcode_in_cells_{chamber}_{roi}.pkl"
        if not redo and target.exists():
            print(f"    Reloading already processed roi")
            barcode_in_cells  = pd.read_pickle(target)
        else:
            if skip_not_done:
                print(f"     Skipping {chamber}, {roi}", flush=True)
                continue
            print(f"     Processing {chamber}, {roi}", flush=True)
            if use_slurm:
                err_corr_fname = str(analysis_folder / f"corrected_barcode_spots_edit{edist}.pkl")
                print('       Starting slurm job')
                slurm_folder = Path.home() / "slurm_logs" / (data_path + f"/{chamber}") / "segment_spots"
                slurm_folder.mkdir(parents=True, exist_ok=True)
                job = issa.segment.get_barcode_in_cells(chamber, roi, data_path, err_corr_fname, save_folder=str(analysis_folder), use_slurm=True, slurm_folder=slurm_folder,
                    scripts_name=f"segment_spots_{chamber}_{roi}")
                continue
            try:
                barcode_in_cells = issa.segment.get_barcode_in_cells(chamber, roi, data_path, err_corr, save_folder=analysis_folder)
            except Exception as e:
                print(f"Error processing {chamber}, {roi}: {e}")
                continue
        barcode_in_cells = barcode_in_cells.loc[barcode_in_cells.index != 0].copy()
        barcode_in_cells.index = barcode_in_cells.index.map(lambda x: f"{chamber}_{roi}_{x}")
        all_cells_list.append(barcode_in_cells)
all_cells = pd.concat(all_cells_list)
all_cells.fillna(0, inplace=True)
print(all_cells.shape)
# add missing barcodes
missing = set(err_corr.corrected_bases.unique()) - set(all_cells.columns)
missing_df = pd.DataFrame(index=all_cells.index, columns=list(missing), data=np.zeros((len(all_cells), len(missing))))
all_cells = pd.concat([all_cells, missing_df], axis=1)
print(all_cells.shape)

cnt_array = all_cells.values.astype(int)

In [ ]:
cnt_array = all_cells.astype(int).values
plt.imshow(cnt_array, interpolation='none', aspect='auto')
plt.colorbar()

In [ ]:
bc_cnt = err_corr.corrected_bases.value_counts()
print(bc_cnt.shape)
valid_barcodes = bc_cnt[bc_cnt > 100].index
print(len(valid_barcodes))
all_cells_valid_bc = all_cells[valid_barcodes]
plt.imshow(all_cells_valid_bc, interpolation='none', aspect='auto')

In [ ]:
cnt_array = all_cells_valid_bc.astype(int).values

In [ ]:
cnt_array.shape

In [ ]:
np.sum(cnt_array > 5, axis=0).shape

In [ ]:
a,b = np.histogram(good_cell_per_barcode) 
a/np.sum(a)

In [ ]:
p0 = 0.85
p1 = 0.14
p2 = 1 - (p0+p1)
print(p0, p1, p2)


In [ ]:
(2*p0 + p1)**2 + 4 * p0 * (p0+p1+p2)

In [ ]:
m = ((2*p0 + p1) + np.sqrt((2*p0 + p1)**2 - 4 * p0 * (p0+p1+p2))) / (2 * (p0+p1+p2))
print(m)

In [ ]:
rol_per_cells = np.sum(cnt_array, axis=1)
bar_per_cells = np.sum(cnt_array > 0, axis=1)
bar_per_good_cells = np.sum(cnt_array > 5, axis=1)
cell_per_barcode = np.sum(cnt_array > 0, axis=0)
good_cell_per_barcode = np.sum(cnt_array > 5, axis=0)
rol_per_barcodew = np.sum(cnt_array, axis=0)

fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot(2, 2, 1)
ax.hist(rol_per_cells, bins=np.arange(0, rol_per_cells.max(), 1), log=True)
ax.set_xlabel('Rolonies per cell')
ax = fig.add_subplot(2, 2, 2)
ax.hist(rol_per_cells, bins=np.arange(0, rol_per_cells.max(), 1), log=False)
ax.set_xlabel('Rolonies per cell')
ax = fig.add_subplot(2, 2, 3)
ax.hist(good_cell_per_barcode, bins=np.arange(0, good_cell_per_barcode.max()+1, 1)-0.5)
ax.set_xlabel('Cells per barcode (count > 5)')


In [ ]:
p2 = 0.36 * (1-0.85) **2
p1 = 0.64* 0.15 + 2 *0.36 * 0.85*0.15
print(p2, p1)

In [ ]:
good_cells = cnt_array[cnt_array.max(axis=1)> 5, :]
rol_per_cells = np.sum(good_cells, axis=1)
bar_per_cells = np.sum(good_cells > 0, axis=1)

fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot(1, 2, 1)
ax.hist(rol_per_cells, bins=np.arange(0, rol_per_cells.max(), 1), log=False)
ax.set_xlabel('Rolonies per good cell')
ax = fig.add_subplot(1, 2, 2)
ax.hist(bar_per_cells, bins=np.arange(0, bar_per_cells.max(), 1), log=False)
ax.set_xlabel('Barcode per good cell')
fig.tight_layout()


In [ ]:
# get masks
stitched_mask = issp.pipeline.stitch_registered(data_path + f"/{chamber}", prefix='mCherry_1_masks', roi=roi, projection='corrected')
bigmask = issp.pipeline.segment.get_big_masks(data_path+ f"/{chamber}", roi, stitched_mask, mask_expansion=5)[...,0]

In [ ]:
# filter rolonies
import numpy as np
# df = all_barcode_spots[(all_barcode_spots["chamber"] == chamber) & (all_barcode_spots["roi"] == roi)]
df = err_corr[(err_corr["chamber"] == chamber) & (err_corr["roi"] == roi)]


In [ ]:
ylim = [16000,8000]
xlim = [12000,18000]
plt.figure(figsize=(20, 20))

# mask with nan for display
m = bigmask.astype(float)
m[bigmask == 0] = np.nan


plt.imshow(get_stack_part(m, xlim, ylim), interpolation='none', alpha=0.5, cmap='prism', extent=[xlim[0], xlim[1], ylim[0], ylim[1]])
sp = get_spot_part(df, xlim, ylim)
plt.scatter(sp["x"], sp["y"],alpha=0.5, s=1, color='k')


In [ ]:
cell_df = issp.pipeline.segment.make_cell_dataframe(data_path+ f'/{chamber}', roi, masks=bigmask, mask_expansion=0, atlas_size=10)
cell_df.head()

In [ ]:
analysis_folder

In [ ]:
cell_df.to_pickle(analysis_folder / f"cell_df_{chamber}_{roi}.pkl")

In [ ]:
barcode_in_cells = issp.segment.cells.count_spots(df.copy(), grouping_column='corrected_bases', masks=bigmask[...,0])
barcode_in_cells.head()

In [ ]:
cell_df.index.name = 'mask_id'
cell_barcode 

In [ ]:
bcm = barcode_in_cells.values[1:,:]
plt.figure(figsize=(7, 2))
plt.suptitle(f"Chamber {chamber}, roi {roi}")
plt.subplot(1, 3, 1)
plt.hist(bcm.sum(axis=1), bins=np.arange(0,30,2))
plt.ylabel('Number of cells')
plt.xlabel('Number of spots\n(all barcodes)')
plt.subplot(1, 3, 2)
plt.hist(bcm.max(axis=1), bins=np.arange(0,30,2))
plt.ylabel('Number of cells')
plt.xlabel('Number of spots\n(most abundant barcode)')
plt.subplot(1, 3, 3)
plt.hist(np.sum(bcm != 0, axis=1), bins=np.arange(0,10,1)-0.5)
plt.ylabel('Number of cells')
plt.xlabel('Number of\ndifferent barcodes')
plt.tight_layout()

In [ ]:
plt.scatter(bcm.sum(axis=1), np.sum(bcm != 0, axis=1), c=bcm.max(axis=1))
plt.colorbar()

In [ ]:
ylim = [16000,8000]
xlim = [12000,18000]
plt.figure(figsize=(20, 20))

# mask with nan for display
m = bigmask.astype(float)
m[bigmask == 0] = np.nan


plt.imshow(m, interpolation='none', alpha=0.5, cmap='prism')

plt.scatter(df["x"], df["y"],alpha=0.5, s=1, color='k')
plt.xlim(xlim)
plt.ylim(ylim)


In [ ]:


ops = issp.io.load_ops(data_path + f'/{chamber}')
tile = ops['mcherry_ref_tile']

In [ ]:
# Load the mCherry masks
import numpy as np
import matplotlib.pyplot as plt
processed = issp.io.get_processed_path(data_path)
tname = "_".join([str(t) for t in tile])
masks_file = processed / chamber / "cells" / f"mCherry_1_masks_corrected_{tname}.npy"
masks = np.load(masks_file)
m = masks.astype(float)
m[m == 0] = np.nan
plt.imshow(m, interpolation='None', cmap='prism')

In [ ]:
# Warp into ref coordinages
masks_file = processed / chamber / "cells" / f"mCherry_1_masks_corrected_{tname}.npy"
masks = np.load(masks_file)
warped_mask, bad_pixels = issp.pipeline.stitch.warp_stack_to_ref(masks, data_path+ f'/{chamber}', prefix='mCherry_1', tile_coors=tile, interpolation=0)
wm = warped_mask.astype(float)
wm[wm == 0] = np.nan
plt.imshow(wm[...,0,0], interpolation='None', cmap='prism')

In [ ]:
# get tile corners

tile_origin

In [ ]:
ok = (err_corr.chamber == chamber) & (err_corr['tile'] == '_'.join([str(t) for t in tile]))
bc = err_corr[ok]
tc = issp.pipeline.stitch.get_tile_corners(data_path + f'/{chamber}', prefix=ops['reference_prefix'], roi=tile[0])
tc = tc[tile[1], tile[2]]
tile_origin = tc[:,0]
plt.imshow(wm[...,0,0], interpolation='None', cmap='prism', alpha=0.5)
plt.scatter(bc.x-tile_origin[1], bc.y-tile_origin[0], s=1, color='k', alpha=0.5)

In [ ]:
issp.pipeline.segment.make_cell_dataframe(data_path, roi, masks=None, mask_expansion=5.0, atlas_size=10)